<a href="https://colab.research.google.com/github/PVgames11/Sistema-de-Notas-em-Python/blob/main/trabalhobdter%C3%A7a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3

# Função para criar a tabela alunos
def criar_tabela_alunos(conn):
    conn.execute('''
        CREATE TABLE IF NOT EXISTS alunos (
            matricula INTEGER PRIMARY KEY,
            nome TEXT,
            idade INTEGER,
            sexo TEXT,
            serie TEXT
        )
    ''')
    print("Tabela alunos criada com sucesso!")

# Função para criar a tabela notas
def criar_tabela_notas(conn):
    conn.execute('''
        CREATE TABLE IF NOT EXISTS notas (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            matricula INTEGER,
            trimestre TEXT,
            materia TEXT,
            nota REAL,
            FOREIGN KEY (matricula) REFERENCES alunos(matricula)
        )
    ''')
    print("Tabela notas criada com sucesso!")

# Função para inserir alunos
def inserir_alunos(conn):
    print("Inserção de Alunos:")
    while True:
        matricula = input("Digite matrícula do aluno(ou digite 'fim' para sair): ")
        if matricula.lower() == 'fim':
            break
        nome = input("Digite o nome do aluno:\n")
        idade = input("Digite a idade do aluno:\n")
        sexo = input("Digite o sexo do aluno (Masculino ou Feminino):\n")
        serie = input("Digite a série em que o aluno estuda:\n")

        try:
            conn.execute('INSERT INTO alunos (matricula, nome, idade, sexo, serie) VALUES (?, ?, ?, ?, ?)', (matricula, nome, idade, sexo, serie))
            conn.commit()
            print("Aluno inserido com sucesso no sistema!")
        except sqlite3.IntegrityError as e:
            print("Erro ao inserir o aluno no sistema:", e)

# Função para inserir notas
def inserir_notas(conn):
    print("Inserção de Notas:")
    while True:
        matricula = input("Digite a matrícula do aluno(ou digite 'fim' para sair): ")
        if matricula.lower() == 'fim':
            break
        trimestre = input("Digite o trimestre em que a nota está:\n")
        materia = input("Digite a matéria:\n")
        nota = input("Digite a nota:\n")

        try:
            cursor = conn.cursor()
            cursor.execute('SELECT matricula FROM alunos WHERE matricula = ?', (matricula,))
            aluno = cursor.fetchone()
            if aluno:
                conn.execute('INSERT INTO notas (matricula, trimestre, materia, nota) VALUES (?, ?, ?, ?)', (matricula, trimestre, materia, nota))
                conn.commit()
                print("Nota inserida com sucesso no sistema!")
            else:
                print("Aluno não encontrado.")
        except sqlite3.IntegrityError as e:
            print("Erro ao inserir a nota no sistema:", e)

# Função para remover aluno
def remover_aluno(conn):
    print("Remoção de Aluno:")
    while True:
        matricula = input("Digite a matrícula do aluno a ser removido(ou digite 'fim' para sair): ")
        if matricula.lower() == 'fim':
            break
        try:
            cursor = conn.cursor()
            cursor.execute('SELECT matricula FROM alunos WHERE matricula = ?', (matricula,))
            aluno = cursor.fetchone()
            if aluno:
                conn.execute('DELETE FROM alunos WHERE matricula = ?', (matricula,))
                conn.commit()
                print("Aluno removido com sucesso!")
            else:
                print("Aluno não encontrado.")
        except sqlite3.Error as e:
            print(f"Erro ao remover aluno: {e}")

# Função para remover nota
def remover_nota(conn):
    print("Remoção de Nota:")
    while True:
        matricula = input("Digite a matrícula do aluno para remover a nota (ou digite 'fim' para sair):\n")
        if matricula.lower() == 'fim':
            break
        materia = input("Digite a matéria para remover a nota:\n")
        try:
            cursor = conn.cursor()
            cursor.execute('SELECT matricula FROM alunos WHERE matricula = ?', (matricula,))
            aluno = cursor.fetchone()
            if aluno:
                conn.execute('DELETE FROM notas WHERE matricula = ? AND materia = ?', (matricula, materia))
                conn.commit()
                print("Nota removida com sucesso!")
            else:
                print("Aluno não encontrado.")
        except sqlite3.Error as e:
            print(f"Erro ao remover nota: {e}")

# Função para atualizar dados do aluno
def atualizar_dados_aluno(conn):
    print("Atualizar Dados do Aluno:")
    while True:
        matricula = input("Digite a matrícula do aluno para atualizar seus dados (ou digite 'fim' para sair): ")
        if matricula.lower() == 'fim':
            return

        try:
            cursor = conn.cursor()
            cursor.execute('SELECT matricula, nome, idade, serie FROM alunos WHERE matricula = ?', (matricula,))
            aluno = cursor.fetchone()
            if aluno:
                matricula, nome_atual, idade_atual, serie_atual = aluno
                novo_nome = input(f"Novo nome do aluno ({nome_atual}): ")
                nova_idade = input(f"Nova idade do aluno ({idade_atual}): ")
                nova_serie = input(f"Nova série do aluno ({serie_atual}): ")
                if novo_nome or nova_serie:
                    conn.execute('UPDATE alunos SET nome = ?, idade = ?, serie = ? WHERE matricula = ?', (novo_nome, nova_idade, nova_serie, matricula))
                    conn.commit()
                    print("Informações do aluno atualizadas com sucesso!")
            else:
                print("Aluno não encontrado.")
        except sqlite3.Error as e:
            print(f"Erro ao atualizar informações do aluno: {e}")

# Função para atualizar nota
def atualizar_nota(conn):
    print("Atualizar Nota:")
    while True:
        matricula = input("Digite a matrícula do aluno para atualizar a nota (ou digite 'fim' para sair): ")
        if matricula.lower() == 'fim':
            return
        materia = input("Digite a matéria para atualizar a nota:\n")
        novo_valor = input("Digite o novo valor da nota:\n")

        try:
            cursor = conn.cursor()
            cursor.execute('SELECT matricula FROM alunos WHERE matricula = ?', (matricula,))
            aluno = cursor.fetchone()
            if aluno:
                conn.execute('UPDATE notas SET nota = ? WHERE matricula = ? AND materia = ?', (novo_valor, matricula, materia))
                conn.commit()
                print("Nota atualizada com sucesso!")
            else:
                print("Aluno não encontrado.")
        except sqlite3.Error as e:
            print(f"Erro ao atualizar nota: {e}")

# Função para consultar os dados
def consultar_dados(conn, tabela):
    try:
        cursor = conn.cursor()
        if tabela == 'alunos':
            opcao = input("Escolha uma opção:\n1. Mostrar um aluno de cada vez\n2. Mostrar todos os alunos\n")
            if opcao == '1':
                matricula = input("Digite a matrícula do aluno que deseja consultar (ou digite 'fim' para sair):\n")
                if matricula.lower() == 'fim':
                    return
                cursor.execute('SELECT * FROM alunos WHERE matricula = ?', (matricula,))
            elif opcao == '2':
                cursor.execute('SELECT * FROM alunos')
            else:
                print("Opção inválida.")
                return
        elif tabela == 'notas':
            opcao = input("Escolha uma opção:\n1. Mostrar notas de um aluno por matrícula\n2. Mostrar todas as notas\n")
            if opcao == '1':
                matricula = input("Digite a matrícula do aluno para consultar suas notas (ou digite 'fim' para sair):\n")
                if matricula.lower() == 'fim':
                    return
                cursor.execute('SELECT * FROM notas WHERE matricula = ?', (matricula,))
            elif opcao == '2':
                cursor.execute('SELECT * FROM notas')
            else:
                print("Opção inválida.")
                return
        else:
            print("Tabela inválida.")
            return

        rows = cursor.fetchall()
        if not rows:
            print("Nenhum dado encontrado.")
            return
        print(f"Dados da tabela {tabela}:")
        for row in rows:
            print(row)
    except sqlite3.Error as e:
        print(f"Erro ao consultar dados da tabela {tabela}: {e}")



# Função para finalizar o programa
def finalizar_programa():
    global continuar
    continuar = False
    print("Programa finalizado.")

# Função menu do programa
def exibir_menu():
    print("\n Seja Bem - Vindo ao sistema de notas.Qual opção você deseja acessar?:")
    print("\n1. Criar tabela de alunos")
    print("2. Criar tabela de notas")
    print("3. Inserir alunos")
    print("4. Inserir notas")
    print("5. Remover aluno")
    print("6. Remover nota")
    print("7. Atualizar dados do aluno")
    print("8. Atualizar notas")
    print("9. Consultar dados")
    print("10. Finalizar programa")

# Conexão com o banco de dados
conn = sqlite3.connect('alunos.db')
print("Conexão Aberta!")

# Loop principal do programa
continuar = True
while continuar:
    exibir_menu()
    opcao = input("Escolha uma opção: ")

    if opcao == '1':
        criar_tabela_alunos(conn)
    elif opcao == '2':
        criar_tabela_notas(conn)
    elif opcao == '3':
        inserir_alunos(conn)
    elif opcao == '4':
        inserir_notas(conn)
    elif opcao == '5':
        remover_aluno(conn)
    elif opcao == '6':
        remover_nota(conn)
    elif opcao == '7':
        atualizar_dados_aluno(conn)
    elif opcao == '8':
        atualizar_nota(conn)
    elif opcao == '9':
        tabela = input("Digite o nome da tabela para consultar os dados (alunos ou notas): ")
        consultar_dados(conn, tabela)
    elif opcao == '10':
        finalizar_programa()
    else:
        print("Opção inválida. Por favor, escolha uma opção válida.")

# Fecha a conexão com o banco de dados
conn.close()


Conexão Aberta!

 Seja Bem - Vindo ao sistema de notas.Qual opção você deseja acessar?:

1. Criar tabela de alunos
2. Criar tabela de notas
3. Inserir alunos
4. Inserir notas
5. Remover aluno
6. Remover nota
7. Atualizar dados do aluno
8. Atualizar notas
9. Consultar dados
10. Finalizar programa
Escolha uma opção: 1
Tabela alunos criada com sucesso!

 Seja Bem - Vindo ao sistema de notas.Qual opção você deseja acessar?:

1. Criar tabela de alunos
2. Criar tabela de notas
3. Inserir alunos
4. Inserir notas
5. Remover aluno
6. Remover nota
7. Atualizar dados do aluno
8. Atualizar notas
9. Consultar dados
10. Finalizar programa
Escolha uma opção: 2
Tabela notas criada com sucesso!

 Seja Bem - Vindo ao sistema de notas.Qual opção você deseja acessar?:

1. Criar tabela de alunos
2. Criar tabela de notas
3. Inserir alunos
4. Inserir notas
5. Remover aluno
6. Remover nota
7. Atualizar dados do aluno
8. Atualizar notas
9. Consultar dados
10. Finalizar programa
Escolha uma opção: 3
Inserção de